# Milestone 3 - Report
------
In this notebook we are going to summarize what was done in Milestone 2 and explain all that was done for the final Milestone. The focus at this point of the project was placed on constructing the awareness model using our different metrics and the final datastory. 

In [15]:
import os
import json
import time
import folium
import branca
import jenkspy
import pickle
import unicodedata
import collections
import numpy as np
import pandas as pd
from scipy import spatial
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 0. Abstract

*What's the motivation behind your project? A 150 word description of the project idea, goals, dataset used. What story you would like to tell and why?*

Major events happen on a regular basis all around the world, some involving high number of casualties but the resulting reaction on the international scale is often far from proportional. Most of the time the largest reaction comes from the place where the incident occurred or places which are closeby. The objective would be to create an awareness map, and determine why people react to an event. From that we would attempt to define an awareness metric. We want to see how factors other than physical proximity come into play such as country, culture, language, religion. With this we could determine which country has the highest level of international awareness. The project would require the Twitter API to acquire hashtag specific tweets with geolocation and therefore measure the awareness and reactions of different communities to a given event. GDELT would be used to recover standardised information regarding different events.
____
____
____


# 1. Recapitulation of Milestone 2

To recap what was done in the previous milestone, here we present a brief summary of the previous notebook which can be found here **(TODO : mettre le lien)**. 

____

## 1.1 Tweet Acquisition
A lot of time was spent acquiring the Tweets for two main reasons : 
- The Twitter dump did not contain the geolocations of the tweets or the users and only containecd 10% of all tweets for a given period. Given that users to not necessarily give their location we wanted to avoid cases where we would not have enough overall located tweets. 
- The Twitter API does not permit the recovery of tweets which are over 1 week old when searching for tweets with a given hashtag around a specific date. 

That is why a significant amount of time was put into recovering the tweets through webscraping using phantom js with random scrolling time and random breaks to avoid being blocked by Twitter.

The tweets were acquired based on their hashtags, all the while making sure to have a wide range of hashtags in different languages. 

**TODO : put the link towards the file containing all the hashtags**

____

## 1.2. Tweet Geolocalisation

Once the tweets were recovered we only kept those for which we had either the geolocalisation of the tweets itself or of the user. As the locations are manually input, they are not standardized and the corresponding location needed to be determined. 

The first step was therefore to format the strings by removing all special characters as setting everything to lower case for example.  Afterwards, we created multiple mappings which were used in the order presented as they represent the confidence in the given method : 
- A country mapper which uses multiple variations of the country name (different spellings and languages) to determine the country it corresponds to with the given ISO2 code
- A capital mapper which maps the capital of a country to the ISO2 code
- A first city mapper where duplicates were removed as there are multiple cities in the world with the same name (example : Torronto in Canada, the US and Australia). 
- A final city mapper with all the cities of all the countries in the world using the database provided here **(TODO : mettre le lien)**

** TODO : put the image of the bird with the number of tweets** 


## 1.3. Visualization of the Reaction For An Event

We then observed for one of the selected events the number of related tweets on a Chloropleth map. This helped us note certain issues in the mappings (high number of tweets in Antarctica for example). 

____


## 1.4. Enriching the Data 

To create our awareness model we wanted to recover information which could be used to determine a sort of distance between the different countries, that is why we recovered for all countries: 
- area
- ISO2
- ISO3
- languages
- border countries
- latitude and longitude
- language_codes
- Internet users
- population
- GDP
- GDP per capita
- religions
- government types
- population in poverty
- unemployment rate

As you can see, we have more features than we had originally presented in Milestone 2. That's because we realized that one of the datasets we used had a lot more (and more complete) information that was of our interest. After all, the 4 datasets that we were using originally ended up being just two, which are these:

- Dataset 1: https://github.com/mledoze/countries
- Dataset 2: https://github.com/opendatajson/factbook.json

To see all the detailed process of the extraction of the data see "Data enriching.ipynb".
____
____
____


____
____
____

# 2. Improving the Mappings

One main issue with the mapping up until now is that there were redundancies which were not correctly handled. At the time of Milestone 2 we would just go through the complete mapping sequentially and stop at the first match. As there were duplicates there was no guarantee that the city found was the most coherent. That is why we corrected the mapping to take into account the population of the different cities which was also provided in the GeoNames database http://download.geonames.org/export/dump/. Therefore when there are multiple cities with the same name in different countries, we only keep the city with the largest population as it would be the most probable. 

We also replaced our intermediate city mapping with a mapping containing the top 20'000 cities in the world (and therefore the most probable). With appoximately 200 countries this corresponds to around 100 cities per country. 

# 3. Metrics For the Awareness Model

To create our model we originally planned to construct a graph linking all countries where the weights of the segments would represent the proximity between countries in terms of awareness to major events. 

For that we used the dataset containing all the information regarding the countries and looked to give numeric values to all the different categories mentioned previously and a few new ones: 

1. Languages
2. Governments 
3. Distances between countries :
    - Birds eye view distances 
    - Hop matrix : the number of countries which separate two given countries (**TODO : noter quil y a le souci des distances a traverser au sein des pays qui n'est pas pris en compte, exemple de la Finlande a la Chine qui passe uniquement par la russie**). 
    - Normalized Adj matrix : a matrix which gives the neighboring countries with a normalizing factor to account for multiple countries on the border
    - Flight routes between countries
4. Religion
5. Population, Area, GDP, Internet Users etc...


What needed to be taken into account here is that we have many variables which are categoric. Therefore we needed to find a way to attribute numeric values in order to construct the final graph.

Remark : most of the information was extracted from the CIA factbook as it is a complete and realiable source.  **TODO : put the link**

**TODO / REMARQUE A GARDER EN TETE POUR LA CONCLUSION AU PIRE : voir si le fait que le pays soit très grand avec une grande population qui peut diminuer l'intérêt par rapport à ce qui se passe ailleurs dans le monde**

____

## 3.1. Languages

Languages are representative of culture and most of the time, countries which speak the same language have a given proximity. That is why this metric was used to create the graph. 

Two main possibilities presented themselves :
- binary : if two countries have the same official language then they are linked
- more informed : some languages are similar in the sense of understanding the other language, learning it, etc. That is why we wanted to take into account Linguistic distances. Unfortunately this is a current research topic and there is no database which gives the Linguistic distances between all the official languages of all countries. Certain databases exist linking English to other languages but this is not sufficient for our application. That is why we decided to use another metric, albeit more abstact, which is to say the phylogenetic trees linking languages together. 


Using the phylogenetic trees taken here (**TODO insert nice link**) http://glottolog.org/glottolog/family, we were able to construct a distance metric and compute the distances between all languages as can be seen below. The notebook handling this can be found here (**TODO : mettre le lien du notebook**)

In [ ]:
# TODO METTRE LA MATRICE & HEATMAP DE DISTANCE ENTRE LES LANGUES

From this language distance matrix we were able to compute a standardized distance between all countries. 

The idea is the following :
- if countries have an official language in common, set the distance to 0
- else get the distances between the official languages and compute the average of the non infinite distances. 
- else set the distance to inf


In [ ]:
# TODO METTRE LA MATRICE DISTANCE / HEATMAP ENTRE LES PAYS SELON LA LANGUE

The issue with this type of distance is that it takes discrete values which don't account for the actual distance between the different languages in terms of comprehension, facility to learn the other language etc.. For example the distance between French and a dialect ....
**TODO : afficher deux valeurs de distance similaires mais qui ne sont pas représentatifs**

Results more representative of the reality of the situation could have been obtained had we had not only the phylogenetic links between the languages but the moment that the a new language appeared from a mother language. Incoroporating this time component would have given a more precise approximation, assuming that languages evolve at similar rates and that once the new language has emerged that there is minimal contact / influence from the mother or any sister languages. 

____

## 3.2. Governments

The type of government is also a good representative of the people's perception to worldwide events. People of different countries with the same government type will be influenced in a similar way. 

After getting the list of all the government types (see "Data enriching.ipynb" for more details on how we got them) we have to convert them to a numeric scale with a meaning.

First, we start by dividing all the government types in three groups and assigning them numerical values -1, 0 and 1. These are the resulting groups:
- Group 1:
    - 'parliamentary democracy'
    - 'parliamentary republic'
    - 'federal republic'
    - 'federation of monarchies'
    - 'semi-presidential republic'
    - 'semi-presidential federation'
- Group 2: 
    - 'non-self-governing overseas territory'
    - 'in transition'
    - 'unknown'
- Group 3: 
    - 'presidential republic'
    - 'presidential democracy'
    - 'monarchy'
    - 'theocratic republic'
    - 'communist state'
    - 'absolute monarchy'

As our criteria to group the government types, we used the variable of the power that the leaders of the government have. Coutries where the leader has a lot of power go in group 3 and will be assigned to the value of 1, the others in group 1 and they will be assigned the value -1. For the non well defined government types, we placed them at the center of the scale with the value of 0.

Here is the actual implementation (full code on "Data enriching.ipynb"):

In [3]:
# Reading the dataframe
data = pd.read_pickle('./DataEnriching/data.pickle')

# Defining the three groups
gov_type_array = [['parliamentary democracy', 'parliamentary republic', 'federal republic', 'federation of monarchies',
                   'semi-presidential republic', 'semi-presidential federation'],
                  ['non-self-governing overseas territory', 'in transition', 'unknown'],
                  ['presidential republic', 'presidential democracy', 'monarchy', 'theocratic republic', 'communist state',
                   'absolute monarchy']]

# Creating the mapping dictionary
mapping_gov_type=dict()
for i, gov_group in enumerate(gov_type_array):
    for gov in gov_group:
        if i==0:
            mapping_gov_type.update({gov: -1})
        elif i==1:
            mapping_gov_type.update({gov: 0})
        else:
            mapping_gov_type.update({gov: 1})

# Mapping gov_type values to their numerical value (1, 0, -1)
data['gov_type_num'] = data.gov_type.map(mapping_gov_type)
data[['gov_type', 'gov_type_num']].head()

,gov_type,gov_type_num
,,
name,,
Aruba,parliamentary democracy,-1.0
Afghanistan,presidential republic,1.0
Angola,presidential republic,1.0
Anguilla,parliamentary democracy,-1.0
Åland Islands,unknown,0.0


In order to compute the distance metric between government types, the numeric government type will be used. The distance metric will be defined by the absolute value of the subtraction between the values of each pair of countries. That way, we end up with a symetric metric with possible values of 0, 1 and 2.

In [4]:
# Reading the dataframe and setting ISO2 as index (as it will be the useful country code)
data = pd.read_pickle(os.path.abspath(os.path.join(os.getcwd() + '\DataEnriching\data.pickle')))
data.reset_index(inplace=True)
data.set_index('ISO2', inplace=True)

In [5]:
# Selecting numeric government type column
gov_type_df = data[['gov_type_num']]

# Creating the matrix with the distances
gov_distance_df = pd.DataFrame(columns=gov_type_df.index.tolist())

for country1, value1 in zip(gov_type_df.index.tolist(), gov_type_df['gov_type_num']):
    row = []
    for country1, value2 in zip(gov_type_df.index.tolist(), gov_type_df['gov_type_num']):
        row.append(abs(value1-value2))
    
    
    dictionary = dict(zip(data.index.tolist(), row))
    gov_distance_df = gov_distance_df.append(dictionary, ignore_index=True)
    
gov_distance_df.index = gov_type_df.index
gov_distance_df.head(10)

,AW,AF,AO,AI,AX,AL,AD,AE,AR,AM,...,VG,VI,VN,VU,WF,WS,YE,ZA,ZM,ZW
ISO2,,,,,,,,,,,,,,,,,,,,,
AW,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,...,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
AF,2.0,0.0,0.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,...,2.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0
AO,2.0,0.0,0.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,...,2.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0
AI,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,...,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
AX,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
AL,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,...,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
AD,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,...,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
AE,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,...,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
AR,2.0,0.0,0.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,...,2.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0


____

## 3.3. Distances 

Distance tends to play an important role in determining people's interest regarding a specific event. But there are several types of distances which can be considered when speaking of countries. 

The first and most evident is the **birds-eye-view distance** between the countries. This is computed based on the positions of the center of each country. But this is not enough. Take for example the US and Canada, the distance between the center of thees two countries is much larger than the distance between France and any of its neighboring countries. 

So what else could we consider to have a more representative representation of distance between the different countries? We came up with three other distance metrics which combined would be more complete.


1. **Relative Importance of Neighbors** :  To take into account the fact that two countries can be neighbors and still have a big distance between them we decided to create a metric which would give importance to countries which are direct neighbors. We also wanted to make sure to give each neighbor the importance it is due. For example France has  multiple countries at its border. But that does not mean that each of these countries are of similar importance. That is why we weighted this metric by the size of the countries. Therefore a small neighboring country such as Luxemburg would have a smaller weight than Germany for example.  

2. **Hop Distance** : This second metric accounts not only for direct neighbors but also for the smallest number of countries which would need to be traversed to connect any two countries in the world. 

3. **Flight Routes** : This last metric accounts for movement of populations between the different countries. The assumption is that the existence of flight routes is due to the fact that people exhibit a certain interest for the other country. The more often you visit a place, the more likely you are to be interested in what is going on there. 


These metrics are all constructed in the country distances noteboook in the GeoMetrics folder (**TODO : put link **). Refer to this noteboook for more detail on how the metrics were constructed. 

** Bird's Eye View Distance / Real Distance **

Constructing this mapping was relatively straightforward. Using the position in latitude and longitude of the countries the distance between all countries in computed all the while being carefull with the extreme values of longitude. It is important to consider the shortest path as the earth is round. 

** TODO : load the pickle file and plot the heatmap as well as the distribution of values in KILOMETERS! (factor 105)**


** Hop Distance **


For this metric as well as for the following we established an adjacency matrix of the connected countries in term of borders. It is important to note that Islands and certain continents are isolated which means that the resulting graph will not be fully connected. 

This will also be a usefull tool to create a neigborhood influence matrix.
We can visualize the graph using networkx. 

**TODO : put the graph from Adrian's notebook with the nodes placed in the position of the countries**

With the obtained graph in networkx we compute the shortest path between all pairs of nodes to obtain what we call the hop matrix between all countries. Countries which are not connected have a hop distance of infinity between them in the final dataframe. 

** TODO : insert the heatmap of the final hop matrix**

**TODO : put a screenshor of the map for a given country + LINK to the html with the map**



** Relative Importance of Neighbors **

The adjacency matrix established previously is then used to create a weighted adjacency matrix where the edges weighted by the size of the neighboring country over the sum of the sizes of all the neighbors. The graph is no longer an undirected graph. 

**TODO : put a screenshot of the map for a given country and link **


** Flight Routes **

Given a dataset of 59036 routes between 3209 airports on 531 airlines in the world as of January 2012. In addition the that we are able so associate all the Airports to their country thanks to a second dataset. **TODO : put the links towards the datasets**

Once that was done, we associated all the routes with the departure and arrival countries. That way were able to determine for each country the proportion of flights to all other countries. 

In the same way as before the graph is a directed graph.

**TODO : put the heatmap**

**TODO : put the graph with the nodes at the positions of the countries**

**TODO : put a screenshor of the map for a given country**

**TODO : put the link towards the interactive map for all the distance metrics**

____

## 3.4. Religions 

Religion, like government, affects the way of thinking of the people. Religion bonds between countries will definitely have an impact on the awareness of events that happen in countries with the same religions.

Here we can see the religion data that we were originally using in Milestone 2:

In [20]:
rel_df_milestone2 = pd.read_pickle("./DataEnriching/Pickles for Milestone 3/final_rel_df.pickle")
rel_df_milestone2.head()

name     religion                              \
                           Christianism Judaism     Islam  Buddhism   
ISO2                                                                  
AF             Afghanistan       0.0003     0.0  0.995600  0.000099   
AO                  Angola       0.8912     0.0  0.010399  0.000099   
AL                 Albania       0.2144     0.0  0.630000  0.000000   
AD                 Andorra       0.9070     0.0  0.008999  0.000000   
AE    United Arab Emirates       0.0714     0.0  0.674800  0.003500   

                                                                              \
     Zoroastrian   Hindu      Sikh Shinto    Baha'i Taoism Jain Confucianism   
ISO2                                                                           
AF      0.000099  0.0003  0.000099    0.0  0.000099    0.0  0.0          0.0   
AO      0.000000  0.0000  0.000000    0.0  0.000099    0.0  0.0          0.0   
AL      0.000000  0.0000  0.000000    0.0  0.002200    0.0  0.0          0.0   
AD      0.000000  0.0035  0.000000    0.0  0.000000    0.0  0.0          0.0   
AE      0.000000  0.2225  0.000000    0.0  0.009999    0.0  0.0          0.0   

                                                                          
     Syncretic religions Animist religions Non-religious Other religions  
ISO2                                                                      
AF                   0.0          0.000099      0.002000          0.0014  
AO                   0.0          0.075899      0.017899          0.0044  
AL                   0.0          0.000000      0.150700          0.0027  
AD                   0.0          0.000000      0.079600          0.0009  
AE                   0.0          0.000000      0.013599          0.0041

Unfortunately, there were too many countries for which the religion information was not complete. That is why we went back to the factbook which was more complete. 

### Methodology
From now on, we will explain how we took the data from the factbook dataset.

We could extract the religions from Dataset 2, but we had to do some cleaning to have the desired format of the data. First we needed to know which where the religions that were in the dataset, so we created an array containing the unique names of the religions that appeared througouth each country. Once we know all the religions, we created the religions dataframe, containing the percentage of every religion in every country. Finally, we grouped the small religions in 10 broader categories of religions.

As said, we start by creating the unique religions array. Here is a sample of part of the array:

In [21]:
#unique_religions[:20]
['Adventist', 'Animist', 'Armenian Apostolic', 'Assembly of God', 'Awakening Churches/Christian Revival', 'Badimo',
 "Baha'i", 'Baptist', 'Bektashi', 'Buddhism', 'Buddhist', 'Bukot nan Jesus', 'Calvinist', 'Cao Dai', 'Catholic', 'Christian',
 'Christianity','Church of England', 'Church of Ireland', 'Church of Norway']

['Adventist',
 'Animist',
 'Armenian Apostolic',
 'Assembly of God',
 'Awakening Churches/Christian Revival',
 'Badimo',
 "Baha'i",
 'Baptist',
 'Bektashi',
 'Buddhism',
 'Buddhist',
 'Bukot nan Jesus',
 'Calvinist',
 'Cao Dai',
 'Catholic',
 'Christian',
 'Christianity',
 'Church of England',
 'Church of Ireland',
 'Church of Norway']

Now that we have all possible religions in one array, we will create the dataframe (rel_df), with all religions as columns, and the country as the index. The cell values will represent the percentage of the religion in the country.

Some of the countries had bad formated data to be extracted (i.e. percentages given by ranges like 10-30%, some countries didn't have the percentages, etc). We manually added the rows of the countries that had issues. For the countries that didn't have percentages, we searched on Wikipedia for the percentages and we also added them manually.

To create the rel_df dataframe, the rows were appended into an initial empty dataframe. The rows were created by looping through all unique_religions and checking if the country in particular had that religion. If it had it, we inserted the percentage. If it didn't have it, we inserted the value zero in that religion column.

Here is what the rel_df looks like, after dropping the smallest religions (columns) that didn't have more than 10% in any country:

In [22]:
rel_df = pd.read_pickle('./DataEnriching/rel_df.pickle')
rel_df.set_index('ISO2', inplace=True)
rel_df.head()

,Armenian Apostolic,Assembly of God,Awakening Churches/Christian Revival,Buddhism,Buddhist,Calvinist,Catholic,Christian,Church of Norway,Congregational Christian Church,...,none,none or other,other,other Christian,other and unspecified,other or none,other or unspecified,unaffiliated,unaffiliated or other,unspecified
ISO2,,,,,,,,,,,,,,,,,,,,,
DZ,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,...,0.000,0.0,0.010,0.000,0.0,0.0,0.0,0.0,0.0,0.000
AO,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,...,0.123,0.0,0.086,0.000,0.0,0.0,0.0,0.0,0.0,0.000
BW,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.791,0.0,0.0,...,0.152,0.0,0.014,0.000,0.0,0.0,0.0,0.0,0.0,0.003
BJ,0.0,0.0,0.0,0.0,0.0,0.0,0.255,0.000,0.0,0.0,...,0.058,0.0,0.026,0.095,0.0,0.0,0.0,0.0,0.0,0.000
BI,0.0,0.0,0.0,0.0,0.0,0.0,0.621,0.000,0.0,0.0,...,0.000,0.0,0.036,0.000,0.0,0.0,0.0,0.0,0.0,0.079


### Grouping religions under broader categories
We grouped the religions in broader categories by adding the percentages. We had to dropp some columns that didn't provide any information about the religious similarity between two countries (i.e unspecified, none or other, other, etc).

In [23]:
# Here are the broader categories that we came up with
protestants = ['Calvinist', 'Church of Norway', 'Congregational Christian Church', 'Ekalesia Niue', 'Evangelical', 
               'Evangelical Lutheran', 'Evangelical Lutheran Church of Iceland', 'Evangelical or Protestant', 'Lutheran',
               'Protestant', 'Protestant and other', 'Seventh-Day Adventist', 'non-Catholic Christians', 'Armenian Apostolic', 
               'Assembly of God', 'Christian', 'Kimbanguist', 'Mormon', 'Zionist Christian', 'nondenominational']
               
catholic = ['Awakening Churches/Christian Revival', 'Catholic', 'Roman Catholic', 'nominally Roman Catholic']
orthodox = ['Eastern Orthodox', 'Ethiopian Orthodox', 'Greek Orthodox', 'Macedonian Orthodox', 'Orthodox', 'Orthodox Christian',
                'Russian Orthodox', 'Serbian Orthodox']
buddhism = ['Buddhism', 'Buddhist', 'Lamaistic Buddhist']
hindu = ['Hindu', 'Indian- and Nepalese-influenced Hinduism']
jewish = ['Jewish', 'Zionist', ]
muslim = ['Muslim', 'Sunni Muslim']
oriental = ['Shintoism', 'Taoist', 'mixture of Buddhist and Taoist']
other = ['Vodoun', 'eclectic mixture of local religions', 'folk religion', 'indigenous beliefs']
animist = ['animist', 'animist or no religion']
atheist = ['atheist or agnostic', 'no religion', 'non-believer/agnostic', 'non-believers']
unaffiliated = ['unaffiliated', 'unaffiliated or other']

# Those religions will be dropped
dropped_cols = ['Kempsville Presbyterian Church', 'none', 'none or other', 'other',
                'other Christian', 'other and unspecified', 'other or none', 'other or unspecified', 'unspecified',
                'atheist and agnostic']

# Arrays needed to run next cell
final_categories = [protestants, catholic, orthodox, buddhism, hindu, jewish, muslim, oriental, other, animist,
                   atheist, unaffiliated]
final_categories_names = ['protestants', 'catholic', 'orthodox', 'buddhism', 'hindu', 'jewish', 'muslim', 
                           'oriental', 'other', 'animist', 'atheist', 'unaffiliated']

The function that unifies the small religions to broader religion groups, and the resulting dataframe are the following:

In [24]:
def generate_categories_df(dataframe):
    df = pd.DataFrame()
    for category, category_name in zip(final_categories, final_categories_names):
        df[category_name] = dataframe[category].sum(axis=1)
    return df

categorized_df = generate_categories_df(rel_df)
categorized_df.head()

,protestants,catholic,orthodox,buddhism,hindu,jewish,muslim,oriental,other,animist,atheist,unaffiliated
ISO2,,,,,,,,,,,,
DZ,0.000,0.000,0.0,0.0,0.0,0.0,0.990,0.0,0.000,0.0,0.0,0.0
AO,0.381,0.411,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0
BW,0.791,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0
BJ,0.135,0.255,0.0,0.0,0.0,0.0,0.277,0.0,0.116,0.0,0.0,0.0
BI,0.239,0.621,0.0,0.0,0.0,0.0,0.025,0.0,0.000,0.0,0.0,0.0


To compute the distances, the euclidean dot product was applied using the following function, that returns the matrix with the distances between each pair of countries.

In [13]:
rel_distances = spatial.distance.squareform(spatial.distance.pdist(categorized_df,'euclidean'))

In [14]:
# Converting it to a dataframe
rel_distance_df = pd.DataFrame(rel_distances, index=rel_df.index.tolist(), columns=rel_df.index.tolist())
rel_distance_df.head(10)

,DZ,AO,BW,BJ,BI,TD,CG,CD,CM,KM,...,UY,VE,AF,BD,BT,LK,IN,IO,MV,NP
DZ,0.000000,1.137621,1.267194,0.777866,1.172172,0.476006,1.138065,1.064002,0.910891,0.022361,...,1.379166,0.007000,0.140716,1.263309,1.144487,1.164662,0.990000,0.990000,1.250672,0.026000
AO,1.137621,0.000000,0.580535,0.418374,0.254733,0.664930,0.232011,0.156467,0.247931,1.121803,...,0.657056,1.143718,1.057338,0.964330,0.886425,0.976756,0.560430,0.560430,0.987226,1.115069
BW,1.267194,0.580535,0.000000,0.765210,0.831246,0.879648,0.810948,0.707871,0.687791,1.259556,...,1.231276,1.272670,1.195643,1.114240,1.071210,1.116598,0.791000,0.791000,1.129041,1.246987
BJ,0.777866,0.418374,0.765210,0.000000,0.470884,0.338941,0.418418,0.363366,0.232897,0.762309,...,0.774877,0.784287,0.695487,0.888417,0.781279,0.863744,0.416455,0.416455,0.903593,0.754105
BI,1.172172,0.254733,0.831246,0.470884,0.000000,0.714326,0.078549,0.154877,0.306165,1.153407,...,0.404360,1.177941,1.096685,1.029192,0.940524,1.040572,0.665873,0.665873,1.051521,1.150862
TD,0.476006,0.664930,0.879648,0.338941,0.714326,0.000000,0.679580,0.595317,0.436905,0.459959,...,0.981408,0.481991,0.407425,1.009283,0.888114,0.941826,0.634667,0.634667,1.009011,0.454033
CG,1.138065,0.232011,0.810948,0.418418,0.078549,0.679580,0.000000,0.142032,0.270889,1.119845,...,0.443997,1.144062,1.059312,0.981134,0.893247,0.995275,0.588874,0.588874,1.005482,1.115895
CD,1.064002,0.156467,0.707871,0.363366,0.154877,0.595317,0.142032,0.000000,0.172748,1.046327,...,0.547723,1.069864,0.987766,0.982777,0.889612,0.982495,0.591608,0.591608,1.001999,1.042351
CM,0.910891,0.247931,0.687791,0.232897,0.306165,0.436905,0.270889,0.172748,0.000000,0.894003,...,0.661545,0.916900,0.833598,0.937706,0.835379,0.921675,0.513266,0.513266,0.953356,0.888699
KM,0.022361,1.121803,1.259556,0.762309,1.153407,0.459959,1.119845,1.046327,0.894003,0.000000,...,1.358087,0.026249,0.135355,1.255647,1.135804,1.157574,0.980204,0.980204,1.243286,0.025612


**TODO : insert heatmap of the distances**

**TODO INSERT MAP WITH ALL RELIGIONS**

In the previous cell we can see the visualization of the religion metric on a map. When clicking on countries which are majoritarely christian the distance metric does not account for the proximity between protestant, catholic and orthodox. That is why we decided to merge the three religions (protestants, catholics, orthodox) into one group (christianity). The results can be seen in the following map. 

**TODO INSERT MAP WITH CHRISTIANS TOGETHER**

Here we can see that now two countries majoritarely christians are closer in the religion distance metric, meaning that we solved the problem.


To compute the last map, we had to solve also the issues with the lack of data of some countries. In the categorized_df dataframe there are countries that have null values in every religion. This represents a big issue because when computing the distance between countries, these countries will have distance zero not for their religious proximity, but because we lacked the data. We managed to find the percentages of the main religions in most of the important countries by searching in Wikipedia. Here is an example of some manually inserted values:

In [12]:
# We found the percentages on wikipedia by doing the search on google: percentages of main religions [country]
data.set_value('Greenland', ('religion', 'christianity'), 0.96);
data.set_value('Madagascar', ('religion', 'atheist'), 0.84);
data.set_value('North Korea', ('religion', 'atheist'), 0.643);
data.set_value('Palestine', ('religion', 'muslim'), 0.85);
data.set_value('Palestine', ('religion', 'jewish'), 0.12);
data.set_value('Sudan', ('religion', 'muslim'), 0.97);

____

## 3.5. Population, Area, GDP, Internet users, population in poverty, unemployment rate

Countries can be characterized by general attributes such as population, size, gross domestic product (GDP), poverty line, internet users and so forth. These metrics are interesting because they are in direct link to the number of tweets in a conutry and can be used as normalizing factors. 
         
As we announced previously, by using the factbook dataset for Milestone 3 we were able to extract more features than in Milestone 2. These features are straight forward to extract from Dataset 2 (see "Data enriching.ipynb"). From the data.pickle file we can extract them directly. The result is the following:

In [27]:
other_features = pd.read_pickle('.\DataEnriching\data.pickle')
cols = [('POP',''),('area',''),('gdp',''),('gdp_capita',''),('Internet users',''),('pop_pov',''),('unemployment','')]
other_features = other_features[cols]
other_features.head()

,POP,area,gdp,gdp_capita,Internet users,pop_pov,unemployment
,,,,,,,
name,,,,,,,
Aruba,113648.0,180.0,2.516000e+09,25300.0,99000.0,NaN,0.069
Afghanistan,33332025.0,652230.0,1.840000e+10,2000.0,2690000.0,0.358,0.350
Angola,20172332.0,1246700.0,9.194000e+10,6800.0,2434000.0,0.405,NaN
Anguilla,16752.0,91.0,1.754000e+08,12200.0,12000.0,0.230,0.080
Åland Islands,NaN,1580.0,NaN,NaN,NaN,NaN,NaN


____

##  3.6. Estimating Number of Active Tweeters 

As will be explained in the next portion of the notebook it is important to have a normalizing factor to account for differences in twitter activity between the different country. The ideal thing would have been to have the number of active tweeters per country. A second best would have been to simply have the number of tweeters per country. Unfortunately none of this is available to the public. That is why it was important to create our own estimation of these values to generate the model. 

We had at our disposal the number of tweets worldwide for 8 different events. We used this to compute the average number of tweets per country and used it as a baseline. We were careful to compute the average of tweets for a given country only using the data when there was no event in the given country. In clearer terms we did not take the number of tweets in France during Charlie Hebdo to compute the average number of tweets. We relied on the 7 other events. In the same fashion we did not consider the tweets for france during the event in Belgium as the countries are sufficiently close to generate larger than normal reactions. The inverse was also done for Belgium.  

**TODO : load this data and print the average number of tweets in a week for a given country, show the difference for France with and without Charlie Hebdo to justify why**

____

# 4. Constructing the Awareness Model 

## 4.1. Correlations

## 4.2. Non Negative Matrix Factorization

Expliquer pourquoi ca n'a pas fonctionné

####  Graph Construction and Graph Diffusion

## 4.3. Regression Model To Predict Reaction Levels

____
____
____

# 4. Critical Assessment and Conclusion